# Adjust index returns for inflation to get real returns

In [1]:
library(tidyverse)
library(ggthemes)
library(reshape2)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
wb <- WorldBank()
indices <- Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date


Attaching package: ‘PerformanceAnalytics’

The following obje

### get rate of inflation from the WorldBank data-set

In [2]:
# see example: https://nbviewer.jupyter.org/github/shyams80/plutons/blob/master/docs-R/WorldBank.ipynb

wb$Meta() %>%
    filter(COUNTRY_NAME == "India" & INDICATOR_NAME %like% "CPI%") %>%
    select(INDICATOR_NAME, COUNTRY_KEY, INDICATOR_KEY) %>%
    print(n=Inf)

# Source:   lazy query [?? x 3]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockVizUs]
  INDICATOR_NAME                          COUNTRY_KEY INDICATOR_KEY
  <chr>                                         <int>         <int>
1 CPI Price, nominal                              135             3
2 CPI Price, nominal, seas. adj.                  135             5
3 CPI Price, % y-o-y, nominal, seas. adj.         135             6


In [3]:
# we need the CPI Price, % y-o-y, nominal, seas. adj.
cpiInflation <- wb$TimeSeries() %>%
    filter(COUNTRY_KEY == 135 & INDICATOR_KEY == 6) %>%
    arrange(YEAR) %>%
    select(YEAR, VALUE) %>%
    as.data.frame

print(cpiInflation)

   YEAR     VALUE
1  2018  3.342855
2  2017  3.325906
3  2016  4.948949
4  2015  4.910535
5  2014  6.423176
6  2013 10.100508
7  2012  9.382309
8  2011  8.314037
9  2010 10.360541
10 2009  9.726791
11 2008  8.572874
12 2007  5.991501
13 2006  6.061719
14 2005  3.743798
15 2004  3.853201
16 2003  4.324896
17 2002  5.203442
18 2001  3.684379
19 2000  5.989490
20 1999  5.905776
21 1998  4.473903
22 1997  5.648696
23 1996  8.897228
24 1995 11.339475
25 1994  6.339953
26 1993  6.364231
27 1992 11.793300
28 1991 13.864358
29 1990  8.965143
30 1989  3.274692
31 1988  9.395963
32 1987  8.793766


### get NIFTY 50 total-return index returns from the Indices data-set

In [4]:
nifty50Dt <- indices$NseTimeSeries() %>%
    filter(NAME == 'NIFTY 50 TR') %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

annRets <- 100*annualReturn(xts(nifty50Dt$CLOSE, nifty50Dt$TIME_STAMP))

df1 <- data.frame(annRets)
df1$YEAR <- year(index(annRets))

print(df1)

           yearly.returns YEAR
1999-12-30      24.398669 1999
2000-12-29     -13.364088 2000
2001-12-31     -15.048927 2001
2002-12-31       5.336092 2002
2003-12-31      76.611426 2003
2004-12-31      13.035473 2004
2005-12-30      38.633169 2005
2006-12-29      41.900536 2006
2007-12-31      56.804842 2007
2008-12-31     -51.271464 2008
2009-12-31      77.590783 2009
2010-12-31      19.224497 2010
2011-12-30     -23.807885 2011
2012-12-31      29.434881 2012
2013-12-31       8.072455 2013
2014-12-31      32.899200 2014
2015-12-31      -3.013065 2015
2016-12-30       4.394821 2016
2017-12-29      30.266177 2017
2018-12-31       4.638185 2018
2019-07-12       7.075363 2019


In [5]:
# merge the two

allData <- merge(df1, cpiInflation, by='YEAR')
print(allData)

   YEAR yearly.returns     VALUE
1  1999      24.398669  5.905776
2  2000     -13.364088  5.989490
3  2001     -15.048927  3.684379
4  2002       5.336092  5.203442
5  2003      76.611426  4.324896
6  2004      13.035473  3.853201
7  2005      38.633169  3.743798
8  2006      41.900536  6.061719
9  2007      56.804842  5.991501
10 2008     -51.271464  8.572874
11 2009      77.590783  9.726791
12 2010      19.224497 10.360541
13 2011     -23.807885  8.314037
14 2012      29.434881  9.382309
15 2013       8.072455 10.100508
16 2014      32.899200  6.423176
17 2015      -3.013065  4.910535
18 2016       4.394821  4.948949
19 2017      30.266177  3.325906
20 2018       4.638185  3.342855


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)